In [ ]:
from moralis import evm_api
import os
import json

GETTING ENVIRONMENTAL VARIABLES

In [ ]:
moralis_api_key = os.getenv('MORALIS_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')

In [ ]:
# Wallet address of User I'm using to test
account_address = "0x9966CfE397CeA90D81e1E45F35Fee4723C2dAe6E"

GETTING RECENT NFT PURCHASE

In [99]:
def get_wallet_nft_transfers(account_address, limit):
  # Parameters required by the get_wallet_nft_transfers endpoint
  params = {
    "chain": "eth",
    "format": "decimal",
    "order": "DESC",
    "limit": limit,
    "address": account_address
  }

  wallet_nft_transfers_result = evm_api.nft.get_wallet_nft_transfers(
    api_key=moralis_api_key,
    params=params,
  )

  return wallet_nft_transfers_result

  # Function to get the last bought nft by the user (Moralis doesn't have an endpoint for getting just a user's recent nft purchase exclusively)

def get_last_bought_nft_by_account(account_address, transactions):
    # Filtering transactions based on 'to_address' matching the account_address
    filtered_transactions = [
        txn for txn in transactions if txn['to_address'] == account_address.lower()
    ]
    # Returning the 'to_address' of the most recent transaction if available
    if filtered_transactions:
      return filtered_transactions[0]['token_address'], filtered_transactions[0]['token_id']
    else:
      return None  # Return None if no matching transactions found


In [102]:
def latest_nft_purchase():
    transactions = get_wallet_nft_transfers(account_address, 10)
    token_address, token_id = get_last_bought_nft_by_account(account_address, transactions['result'])
    return token_address, token_id

In [103]:
token_address, token_id = latest_nft_purchase()

In [104]:
print(token_address, token_id)

0x026224a2940bfe258d0dbe947919b62fe321f042 2950


GETTING IMAGE URL OF RECENTLY PURCHASED NFT

In [122]:
def get_multiple_nft_result(address, id):
  body = {
      "tokens": [
          {
              "token_address": address,
              "token_id": id
          },
      ],
      "media_items": True,
      "normalize_metadata": True
  }

  params = {
    "chain": "eth"
  }

  multiple_nft_result = evm_api.nft.get_multiple_nfts(
    api_key=moralis_api_key,
    body=body,
    params=params,
  )

  return multiple_nft_result

In [123]:
def extract_image_url():
    ipfsGateway = "https://ipfs.io/ipfs/"
    ipfs_url = "ipfs://"

    # Parsing the get_multiple_nft_result into a dictionary
    nft_response = get_multiple_nft_result(token_address, token_id)
    metadata = json.loads(nft_response[0]['metadata'])
    image_url = metadata['image']

    # Replace IPFS prefix if necessary
    return image_url.replace(ipfs_url, ipfsGateway) if image_url.startswith(ipfs_url) else image_url